In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score


In [2]:
df = pd.read_csv("./train.csv")

In [3]:
df.head()

,AC,AF,AR,AS,AST,AY,AwayTeam,Date,FTR,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam,league
0,3.0,12.0,0.0,14.0,4.0,0.0,Stuttgart,07/08/09,H,6.0,12.0,0.0,13.0,7.0,0.0,0.0,0.0,Wolfsburg,bundesliga
1,1.0,10.0,0.0,7.0,0.0,1.0,FC Koln,08/08/09,H,16.0,8.0,0.0,24.0,11.0,0.0,0.0,0.0,Dortmund,bundesliga
2,3.0,20.0,0.0,15.0,3.0,2.0,Hannover,08/08/09,H,5.0,16.0,0.0,10.0,4.0,0.0,0.0,3.0,Hertha,bundesliga
3,10.0,28.0,0.0,9.0,3.0,2.0,Bayern Munich,08/08/09,D,3.0,10.0,0.0,9.0,1.0,1.0,1.0,0.0,Hoffenheim,bundesliga
4,5.0,28.0,0.0,13.0,7.0,2.0,Leverkusen,08/08/09,D,3.0,22.0,0.0,8.0,4.0,2.0,1.0,1.0,Mainz,bundesliga


In [4]:
df_cleaned = df.dropna(axis = 0, how ='any') 

In [14]:

X= df_cleaned.drop("FTR",axis=1)
X = X.drop("Date",axis=1)

In [15]:
from sklearn.preprocessing import LabelEncoder




le = LabelEncoder()

In [16]:
X["AwayTeamEnc"] = le.fit_transform(X["AwayTeam"].astype(str))
X["HomeTeamEnc"] = le.fit_transform(X["HomeTeam"].astype(str))
X["leagueEnc"] = le.fit_transform(X["league"].astype(str))
X= X.drop("AwayTeam", axis=1)
X=X.drop("HomeTeam", axis=1)
X= X.drop("league", axis=1)

In [17]:
y = le.fit_transform(df_cleaned.FTR)

In [31]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(y)

In [33]:
X.shape, y.shape , dummy_y.shape

((12776, 17), (12776,), (12776, 3))

In [34]:

def twolayer_model():
	# create model
	model = Sequential()
	model.add(Dense(34, input_dim=17, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [35]:
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=twolayer_model, epochs=200, batch_size=5, verbose=0)

In [36]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True)

In [37]:
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [38]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 65.15% (0.89%)


In [39]:
results

array([0.65343979, 0.63982157, 0.66134337])

In [40]:
estimator

In [42]:
model = twolayer_model()
trainedmodel = model.fit(X, dummy_y, epochs=150, batch_size=10)



Epoch 1/150
12776/12776 [==============================] - 2s 191us/step - loss: 3.0091 - acc: 0.5475
Epoch 2/150
12776/12776 [==============================] - 2s 160us/step - loss: 0.8747 - acc: 0.6106
Epoch 3/150
12776/12776 [==============================] - 3s 212us/step - loss: 0.8459 - acc: 0.6223
Epoch 4/150
12776/12776 [==============================] - 3s 219us/step - loss: 0.8449 - acc: 0.6188
Epoch 5/150
12776/12776 [==============================] - 2s 140us/step - loss: 0.8363 - acc: 0.6296
Epoch 6/150
12776/12776 [==============================] - 2s 179us/step - loss: 0.8306 - acc: 0.6285
Epoch 7/150
12776/12776 [==============================] - 3s 196us/step - loss: 0.8213 - acc: 0.6292
Epoch 8/150
12776/12776 [==============================] - 3s 208us/step - loss: 0.8200 - acc: 0.6270
Epoch 9/150
12776/12776 [==============================] - 3s 234us/step - loss: 0.8250 - acc: 0.6256
Epoch 10/150
12776/12776 [==============================] - 4s 325us/step - loss: 

In [45]:

_, accuracy = model.evaluate(X, dummy_y)
print('Accuracy: %.2f' % (accuracy*100))

12776/12776 [==============================] - 1s 51us/step
Accuracy: 66.70


In [ ]:

predictions = trainedmodel.predict(X)
# round predictions 
rounded = [round(x[0]) for x in predictions]

In [ ]:
# make class predictions with the model
predictions = model.predict_classes(X)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))